# YOLOv8m Self-Training for Object Segmentation in Semi-Supervised Learning

**Welcome to our Kaggle Notebook!** This project showcases the power of YOLOv8m for object segmentation using a self-training approach in the realm of semi-supervised learning. With a limited set of 1200 labeled images and an extensive pool of over 5000 unlabeled data, we embark on a captivating journey to enhance object segmentation using YOLOv8m in the field of computer vision.

**🌟 Key Highlights 🌟**

- **YOLOv8m:** Our model of choice, YOLOv8m, is renowned for real-time object detection and segmentation, making it a powerful tool for our project.
- **Semi-Supervised Learning:** We leverage semi-supervised learning principles to maximize the utility of our scarce labeled data.
- **Self-Training:** Through an iterative self-training process, we gradually expand our labeled dataset by incorporating the model's confident predictions for improved object segmentation.

> **⚠️ Warning - Training Duration:**
> This notebook may take up to 9 hours to train on regular Kaggle GPUs, depending on the number of self training iterations. It is highly recommended to try this out on GPUs that can run without any restrictions, allowing you to experiment with larger models and more iterations for optimal results and in-depth exploration.

> **📣 Note - Competition Submission:**
> Please be aware that the competition submission component has not yet been implemented. It will be added in the near future, so stay tuned for updates.

Join us as we explore the world of object segmentation and semi-supervised learning with YOLOv8m, uncovering valuable insights and practical techniques along the way!


# 1. Importing Libraries

In [ ]:
!pip install ultralytics==8.0.176 -q
!pip install pycocotools -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
os.environ['WANDB_DISABLED'] = 'true'

import shutil
import json
import random
from tqdm import tqdm

from PIL import Image

from ultralytics import YOLO

# 2. Creating Directories

In [ ]:
# Creating Directories

parent_dirpath = "/kaggle/working/yolov8"

os.mkdir(parent_dirpath)
os.mkdir("/kaggle/working/temp_images")
os.mkdir("/kaggle/working/temp_labels")

os.mkdir(os.path.join(parent_dirpath, "train"))
os.mkdir(os.path.join(parent_dirpath, "train", "images"))
os.mkdir(os.path.join(parent_dirpath, "train", "labels"))

os.mkdir(os.path.join(parent_dirpath, "test"))
os.mkdir(os.path.join(parent_dirpath, "test", "images"))
os.mkdir(os.path.join(parent_dirpath, "test", "labels"))


# 3. Defining Functions

- **tiff_to_jpg**: Converts .tiff images to .jpg format and stores them in kaggle's working directory for future use.
- **vertices_to_txt**: Converts COCO labels to YOLO labels.

In [ ]:
def tiff_to_jpg(file_name):
    
    tiff_image_path = "/kaggle/input/hubmap-hacking-the-human-vasculature/train/" + str(file_name) + ".tif"
    tiff_image = Image.open(tiff_image_path)
    destination_path = "/kaggle/working/temp_images/" + file_name + ".jpg"
    tiff_image.save(destination_path, 'JPEG')
    
    return 0

def vertices_to_txt(file_id, annotations, list_of_vertices):
    
    file_contents = []

    for i in range(len(annotations)):

        yolo_format = []
        flag = 1

        if annotations[i]['type'] == 'glomerulus':
            yolo_format.append(str(1))
            flag = 1
        elif annotations[i]['type'] == 'blood_vessel':
            yolo_format.append(str(0))
            flag = 1
        else:
            flag = 0


        if (flag):

            list_of_vertices = annotations[i]['coordinates'][0]
            for vertex in list_of_vertices:
                yolo_format.append(str(vertex[0]/512))
                yolo_format.append(str(vertex[1]/512))

        yolo_format = " ".join(yolo_format)

        file_contents.append(yolo_format)

    file_name = "/kaggle/working/temp_labels/" + str(file_id) + ".txt"

    with open(file_name, "w") as file:
        if (len(file_contents) == 0):
            pass
        elif (len(file_contents) == 1):
            file.write(str(file_contents[-1]))
        else:
            for k in range(len(file_contents)-1):
                file.write(str(file_contents[k]) + "\n")

            file.write(str(file_contents[-1]))
            
    return 0

# 4. Data Processing

In [ ]:
train_filepath = "/kaggle/input/hubmap-hacking-the-human-vasculature/train"

all_images = os.listdir(train_filepath)
print("No. of images:", len(all_images))

In [ ]:
# Use the above-defined functions to process the data.

json_filepath = "/kaggle/input/hubmap-hacking-the-human-vasculature/polygons.jsonl"
file_ids = []

with open(json_filepath, 'r') as file:
    
    for line in file:
        data = json.loads(line)
        file_id = data['id']
        annotations = data['annotations']
        list_of_vertices = annotations[0]['coordinates'][0]
        tiff_to_jpg(file_id)
        vertices_to_txt(file_id, annotations, list_of_vertices)
        file_ids.append(file_id)

In [ ]:
# Create a list of all file-ids which are unlabelled.

all_file_ids = []

for file_name in all_images:
    file_name = file_name.split('.')
    all_file_ids.append(file_name[0])

unlabelled_images = list(set(all_file_ids).difference(file_ids))

In [ ]:
# Randomly split the labelled dataset into train and test sets with a ratio of 0.8.

random.shuffle(file_ids)

# Create the Train directory
for i in range(0, int(0.8*len(file_ids))):
    
    old_path_img = "/kaggle/working/temp_images/" + str(file_ids[i]) + ".jpg"
    new_path_img = "/kaggle/working/yolov8/train/images/" + str(file_ids[i]) + ".jpg"
    shutil.copy(old_path_img, new_path_img)
    
    old_path_txt = "/kaggle/working/temp_labels/" + str(file_ids[i]) + ".txt"
    new_path_txt = "/kaggle/working/yolov8/train/labels/" + str(file_ids[i]) + ".txt"
    shutil.copy(old_path_txt, new_path_txt)

# Create the Test directory
for i in range(int(0.8*len(file_ids)), len(file_ids)):
    
    old_path_img = "/kaggle/working/temp_images/" + str(file_ids[i]) + ".jpg"
    new_path_img = "/kaggle/working/yolov8/test/images/" + str(file_ids[i]) + ".jpg"
    shutil.copy(old_path_img, new_path_img)
    
    old_path_txt = "/kaggle/working/temp_labels/" + str(file_ids[i]) + ".txt"
    new_path_txt = "/kaggle/working/yolov8/test/labels/" + str(file_ids[i]) + ".txt"
    shutil.copy(old_path_txt, new_path_txt)
    
shutil.rmtree("/kaggle/working/temp_images")
shutil.rmtree("/kaggle/working/temp_labels")

In [ ]:
# Create the custom_config.yaml file for YOLO model.

with open("/kaggle/working/custom_config.yaml", "w") as file:
    file.write("path: /kaggle/working/yolov8" + "\n")
    file.write("train: train/images" + "\n")
    file.write("val: test/images" + "\n")
    file.write("test: test/images" + "\n")
    file.write("nc: 2" + "\n")
    file.write("names: ['blood_vessel','glomerulus']")

In [ ]:
# Create a list of all file-ids in the test set.

test_set_file_ids = []
test_set_filepath = "/kaggle/working/yolov8/test/images"
test_set_images_filepaths = os.listdir(test_set_filepath)

for file_name in test_set_images_filepaths:
    file_name = file_name.split('.')
    test_set_file_ids.append(file_name[0])
        
len(test_set_file_ids)

# 5. Self training

Self-training is an iterative approach where a model is trained on a small amount of labeled data, and then it's used to make predictions on unlabeled data.

- **Pseudo-Labeling**: Predictions on unlabeled data are used to create pseudo-labels, treating these predictions as if they were true labels for the unlabeled data.

- **Confidence Thresholding**: Typically, a confidence threshold is applied to select only highly confident predictions, reducing the risk of including incorrect labels. Here the confidence score is set to **0.4** as it gives very good results at this score.

- **Iterative Refinement**: The model's pseudo-labeled dataset is gradually expanded and refined over multiple iterations, allowing the model to learn from its own predictions. We set the number of iterations to **10**.

- **Benefits**: Self-training can improve model performance in semi-supervised learning by leveraging the large pool of unlabeled data, making it a resource-efficient approach for training machine learning models.

- **Risks**: Incorrect predictions with high confidence are used as pseudo-labels, potentially degrading the model's performance over iterations. Careful selection and verification of pseudo-labeled data and setting appropriate confidence thresholds are essential to mitigate this risk.

In [ ]:
for iteration in range(10):
    
    # Model training, we enable data augmentation
    model = YOLO('yolov8m-seg.pt')
    results = model.train(data='/kaggle/working/custom_config.yaml',
                          epochs=15, imgsz=512, optimizer='Adam',
                          seed=42, close_mosaic=0, mask_ratio=1, val=True,
                          degrees=90, translate=0.1, scale=0.5, flipud=0.5, fliplr=0.5)

    images_added = 0
    
    # Update unlabelled image list here.
    
    # 1. Get all file_ids from train_set
    train_set_file_ids = []
    train_set_filepath = "/kaggle/working/yolov8/train/images"
    train_set_images_filepaths = os.listdir(train_set_filepath)

    for file_name in train_set_images_filepaths:
        file_name = file_name.split('.')
        train_set_file_ids.append(file_name[0])
        
    # 2. Add file_ids from test_set
    all_labelled_image_file_ids = train_set_file_ids + test_set_file_ids
    
    # 3. Find differences from inputs and combined list & create new unlabelled image file_id list.
    unlabelled_images = list(set(all_file_ids).difference(all_labelled_image_file_ids))

    for file_id in tqdm(unlabelled_images):

        # Create a temporary image for prediction
        tiff_image_path = "/kaggle/input/hubmap-hacking-the-human-vasculature/train/" + str(file_id) + ".tif"
        tiff_image = Image.open(tiff_image_path)
        destination_path = "/kaggle/working/temp_image.jpg"
        tiff_image.save(destination_path, 'JPEG')

        results = model.predict(destination_path, verbose=False)

        # This flag determines whether the predicted image is added to the training set. If set to 0, the image will not be added.
        flag = 1
        file_contents = []

        # If no detections, set flag to 0.
        for result in results:
            boxes = result.boxes.conf
            if len(boxes) != 0:
                classes = result.boxes.cls
                masks = result.masks.xyn
            else:
                flag = 0
        
        # If model's confidence score is less than 0.4, set flag to 0.
        if (flag):
            for i in range(len(boxes)):
                if boxes[i] < 0.4:
                    flag=0
                    break

        # Use the image for training, if flag set to 1.            
        if(flag):
            
            # Copy image
            des_img_filepath = os.path.join("/kaggle/working/yolov8/train/images/" + str(file_id) + ".jpg")
            shutil.copy(destination_path, des_img_filepath)

            for i in range(len(boxes)):

                yolo_format = []

                if classes[i] == 1:
                    yolo_format.append(str(1))
                else:
                    yolo_format.append(str(0))

                list_of_vertices = masks[i]
                for vertex in list_of_vertices:
                    yolo_format.append(str(vertex[0]))
                    yolo_format.append(str(vertex[1]))

                yolo_format = " ".join(yolo_format)

                file_contents.append(yolo_format)
            
            # Create YOLO labels
            file_name = os.path.join("/kaggle/working/yolov8/train/labels/" + str(file_id) + ".txt")

            with open(file_name, "w") as file:
                if (len(file_contents) == 1):
                    file.write(str(file_contents[-1]))
                else:
                    for k in range(len(file_contents)-1):
                        file.write(str(file_contents[k]) + "\n")

                    file.write(str(file_contents[-1]))

            images_added += 1

        flag = 1
        del model

    print("Images added to training set:", images_added)    

In [ ]:
print("Final length of Train dataset:", len(list(os.listdir("/kaggle/working/yolov8/train/labels"))))

In [ ]:
# results = model.predict("/kaggle/working/temp_image.jpg")
# count = 0

# for result in results:
#     boxes = result.boxes.conf # confidence scores
#     classes = result.boxes.cls # class in float
#     masks = result.masks.xyn # location of each segment, normalised

In [ ]:
# masks[0].shape

# 6. Final model training
- The model is trained on the final training dataset created after 10 loops of self-training.


In [ ]:
model = YOLO('yolov8x-seg.pt')
results = model.train(data='/kaggle/working/custom_config.yaml',
                      epochs=50, imgsz=512, optimizer='Adam',
                      seed=42, close_mosaic=0, mask_ratio=1, val=True,
                      degrees=90, translate=0.1, scale=0.5, flipud=0.5, fliplr=0.5)

## 7. Competition Submission
- NOT IMPLEMENTED YET

In [ ]:
# import base64
# import numpy as np
# from pycocotools import _mask as coco_mask
# import typing as t
# import zlib

# def encode_binary_mask(mask: np.ndarray) -> t.Text:
#     """Converts a binary mask into OID challenge encoding ascii text."""

#     # check input mask --
#     if mask.dtype != np.bool:
#     raise ValueError(
#         "encode_binary_mask expects a binary mask, received dtype == %s" %
#         mask.dtype)

#     mask = np.squeeze(mask)
#     if len(mask.shape) != 2:
#     raise ValueError(
#         "encode_binary_mask expects a 2d mask, received shape == %s" %
#         mask.shape)

#     # convert input mask to expected COCO API input --
#     mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
#     mask_to_encode = mask_to_encode.astype(np.uint8)
#     mask_to_encode = np.asfortranarray(mask_to_encode)

#     # RLE encode mask --
#     encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

#     # compress and base64 encoding --
#     binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
#     base64_str = base64.b64encode(binary_str)
#     return base64_str

In [ ]:
# test_filepath = "/kaggle/input/hubmap-hacking-the-human-vasculature/test"

# df_submission = pd.DataFrame(columns=['id', 'height', 'width', 'prediction'])

# for file in file_list:
    
#     file_name = file_name.split('.')
#     file_name = file_name[0]
    
#     tiff_image_path = "/kaggle/input/hubmap-hacking-the-human-vasculature/test/" + str(file)
#     tiff_image = Image.open(tiff_image_path)
#     destination_path = "/kaggle/working/temp_images/" + file_name + ".jpg"
#     tiff_image.save(destination_path, 'JPEG')
    
#     results = model.predict(destination_path)
    
#     prediction = []
    
#     for result in results:
#           for i in range(len(result.boxes.cls)):
    #         prediction.append(str(result.boxes.cls[i]))
    #         prediction.append(str(result.boxes.conf[i]))
    #         prediction.append(encode_binary_mask("ndarray_mask"))
    
#     prediction = " ".join(prediction)

#     row1 = pd.Series([file_name, 512, 512, prediction])
#     df_submission.append(row1, ignore_index=True)

# df_submission.tail()

In [ ]:
# df_submission.to_csv("submission.csv", ignore_index=True)

In [ ]:
# for result in results:
#     boxes = result.boxes.conf
#     classes = result.boxes.cls
#     masks = result.masks.xyn